## Importing pandas
 - for data manipulation purpose we use pandas


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Loadind the DataSet
- load the csv file into a pandas DataFrame using read_csv()

In [20]:
raw_data = pd.read_csv('./EV_2010-2025.csv')

In [21]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      3798 non-null   object 
 1   category    3798 non-null   object 
 2   parameter   3798 non-null   object 
 3   mode        3798 non-null   object 
 4   powertrain  3798 non-null   object 
 5   year        3798 non-null   int64  
 6   unit        3798 non-null   object 
 7   value       3798 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 237.5+ KB


In [39]:
raw_data.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


## Cleaning the data
 - remove the unwanted columns like `EV category`, and `powertrain`
 - filtering only the `EV sales` 
 - which is useless col for us because we're going to predict the sales not gonna classify the sales. 

In [25]:
clean = raw_data.drop(columns=['category', 'powertrain'])
clean = clean[clean['parameter'] == 'EV sales']

In [31]:
clean.head()

,region,parameter,mode,year,unit,value
0,Australia,EV sales,Cars,2011,Vehicles,49.0
6,Australia,EV sales,Cars,2012,Vehicles,80.0
9,Australia,EV sales,Cars,2012,Vehicles,170.0
10,Australia,EV sales,Cars,2013,Vehicles,190.0
13,Australia,EV sales,Cars,2013,Vehicles,100.0


In [36]:
mode = clean['mode'].value_counts()
mode

mode
Cars    1342
Name: count, dtype: int64

In [40]:
vehicles_countries_grouped = clean.groupby('region')['value'].sum()

## Importing Machine Learning Library
- from sci-kit learn, import the train_test_split method for sliptting the dataset into two one for `Training the model` another one for `Testing the model`